# Initial Coke Data Analysis and Cleaning

In [437]:
import pandas as pd
import os
import duckdb

In [438]:
pd.set_option('display.max_columns', None)

In [439]:
activity_report = pd.read_csv('/Users/jamesferry/Documents/GitHub/CokeDwellTimeDashboard/data/raw/CokeActivityTracking.csv', encoding='ISO-8859-1', dtype={'Order #': str})
order_report = pd.read_csv('/Users/jamesferry/Documents/GitHub/CokeDwellTimeDashboard/data/raw/CokeOrderView.csv', encoding='ISO-8859-1', dtype={'Shipment #': str, 'SAP Delivery # (Order#)': str})
trailer_report = pd.read_csv('/Users/jamesferry/Documents/GitHub/CokeDwellTimeDashboard/data/raw/CokeTrailerActivity.csv', encoding='ISO-8859-1', dtype={'SHIPMENT_ID': str})

## Activity Tracking Report

In [440]:
activity_report.head()

,ï»¿Create DateTime,User Id,Transaction Type,Transaction Code,Transaction Description,LPN,Item Name,From Location,To Location,Task Id,Pallet Id,From LPN,NBR_UNITS (CS),Order/ASN,Order #,Lot Code
0,"Oct 31, 2024 7:22:57 AM",179710,800,2,Load trailer/ oLPN,100250003447451046,281582.0,NaN,NaN,NaN,NaN,NaN,112,405883529.0,0405883529,0013268156
1,"Oct 31, 2024 7:22:57 AM",179710,800,2,Load trailer/ oLPN,100250003447896717,281587.0,NaN,NaN,NaN,NaN,NaN,112,405883529.0,0405883529,0013268154
2,"Oct 31, 2024 7:27:14 AM",167827,800,2,Load trailer/ oLPN,100250010000728811,289482.0,NaN,NaN,NaN,NaN,NaN,100,405877816.0,0405877816,FA24235-2
3,"Oct 31, 2024 7:27:14 AM",167827,800,2,Load trailer/ oLPN,100250010000867954,289483.0,NaN,NaN,NaN,NaN,NaN,100,405877816.0,0405877816,FA24275-2
4,"Oct 31, 2024 7:27:18 AM",173014,800,2,Load trailer/ oLPN,100250003449818076,370685.0,NaN,NaN,NaN,NaN,NaN,90,405869713.0,0405869713,0013264168


In [441]:
activity_report.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2900 entries, 0 to 2899
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ï»¿Create DateTime       2900 non-null   object 
 1   User Id                  2900 non-null   object 
 2   Transaction Type         2900 non-null   int64  
 3   Transaction Code         2900 non-null   int64  
 4   Transaction Description  2900 non-null   object 
 5   LPN                      2900 non-null   int64  
 6   Item Name                2839 non-null   float64
 7   From Location            500 non-null    object 
 8   To Location              1319 non-null   object 
 9   Task Id                  39 non-null     object 
 10  Pallet Id                113 non-null    object 
 11  From LPN                 0 non-null      float64
 12  NBR_UNITS (CS)           2900 non-null   int64  
 13  Order/ASN                2867 non-null   float64
 14  Order #                 

In [442]:
activity_report.isnull().sum()

ï»¿Create DateTime            0
User Id                       0
Transaction Type              0
Transaction Code              0
Transaction Description       0
LPN                           0
Item Name                    61
From Location              2400
To Location                1581
Task Id                    2861
Pallet Id                  2787
From LPN                   2900
NBR_UNITS (CS)                0
Order/ASN                    33
Order #                       0
Lot Code                     33
dtype: int64

In [443]:
activity_report.describe()

,Transaction Type,Transaction Code,LPN,Item Name,From LPN,NBR_UNITS (CS),Order/ASN
count,2900.0,2900.0,2.900000e+03,2839.000000,0.0,2900.000000,2.867000e+03
mean,800.0,2.0,9.611207e+16,363908.304333,NaN,87.158966,3.830630e+08
std,0.0,0.0,1.992002e+16,72136.860861,NaN,17.141335,6.439205e+07
min,800.0,2.0,2.500190e+14,243782.000000,NaN,11.000000,2.014284e+08
25%,800.0,2.0,1.002500e+17,369578.000000,NaN,90.000000,4.058814e+08
50%,800.0,2.0,1.002500e+17,369593.000000,NaN,90.000000,4.058835e+08
75%,800.0,2.0,1.002500e+17,370693.000000,NaN,90.000000,4.058843e+08
max,800.0,2.0,1.002500e+17,702301.000000,NaN,180.000000,4.058880e+08


In [444]:
print(activity_report.columns)

Index(['ï»¿Create DateTime', 'User Id', 'Transaction Type', 'Transaction Code',
       'Transaction Description', 'LPN', 'Item Name', 'From Location',
       'To Location', 'Task Id', 'Pallet Id', 'From LPN', 'NBR_UNITS (CS)',
       'Order/ASN', 'Order #', 'Lot Code'],
      dtype='object')


In [445]:
activity_report.rename(columns={'ï»¿Create DateTime': 'Create DateTime', 'Order #': 'Order Num'}, inplace=True)

In [446]:
columns_to_keep = ['Create DateTime', 'Order Num']
activity_report = activity_report.drop(columns=activity_report.columns.difference(columns_to_keep))

activity_report.head()

,Create DateTime,Order Num
0,"Oct 31, 2024 7:22:57 AM",0405883529
1,"Oct 31, 2024 7:22:57 AM",0405883529
2,"Oct 31, 2024 7:27:14 AM",0405877816
3,"Oct 31, 2024 7:27:14 AM",0405877816
4,"Oct 31, 2024 7:27:18 AM",0405869713


In [447]:
activity_report['Create DateTime'] = pd.to_datetime(activity_report['Create DateTime'])

In [448]:
order_groups = activity_report.groupby('Order Num')['Create DateTime']

In [449]:
load_times = order_groups.agg(lambda x: round((x.max() - x.min()).total_seconds() / 60, 2)).reset_index()
load_times.columns = ['Order Num', 'Load Time (minutes)']

In [450]:
activity_report['Shift'] = activity_report['Create DateTime'].apply(lambda x: '1' if 7<=x.hour < 19 else '2')

In [451]:
activity_report.head()

,Create DateTime,Order Num,Shift
0,2024-10-31 07:22:57,0405883529,1
1,2024-10-31 07:22:57,0405883529,1
2,2024-10-31 07:27:14,0405877816,1
3,2024-10-31 07:27:14,0405877816,1
4,2024-10-31 07:27:18,0405869713,1


In [452]:
activity_report['Order Type'] = activity_report['Order Num'].apply(lambda x: 'Shuttle' if x.startswith('02') else ('Customer Load' if x.startswith('04') else 'Unknown'))

In [453]:
additional_info = activity_report.groupby('Order Num').agg({'Shift': 'first', 'Order Type': 'first'}).reset_index()
load_times = load_times.merge(additional_info, on='Order Num', how='left')

In [454]:
con = duckdb.connect()

In [455]:
con.register('load_times_df', load_times)
con.execute('CREATE TABLE load_times AS SELECT * FROM load_times_df')

In [456]:
print(load_times)

      Order Num  Load Time (minutes) Shift     Order Type
0    0201428121                29.03     1        Shuttle
1    0201428407               151.58     2        Shuttle
2    0201428681                79.28     2        Shuttle
3    0201428695                96.98     2        Shuttle
4    0201428910                90.30     1        Shuttle
..          ...                  ...   ...            ...
208  0405885725               103.35     1  Customer Load
209  0405886093               178.62     1  Customer Load
210  0405886147               115.12     1  Customer Load
211  0405886256               201.12     2  Customer Load
212  0405888024               125.25     2  Customer Load

[213 rows x 4 columns]


## Order View Report

In [457]:
order_report.head()

,Ref Shipment Nbr,Shipment #,SAP Delivery # (Order#),PO #,Order Status,Order Type,Carrier,Ship To,Order Pick Up Date,Appointment,Appointment Date,Check In DateTime,Appointment Type,Trailer,LOAD_OF_THE_DAY,POOL_SEQ#,Trailer Location,Trailer Status,Appointment Notes,Wave #
0,309186851.0,9010143845,0405510588,817183,Shipped,Customer Orders,CPU,GIANT OF MARYLAND,"Jan 29, 2024 12:00:00 AM",4219595.0,"Jan 29, 2024 10:00:00 AM",01/29/24 11:11,LIVE,10,NaN,1,NaN,NaN,Trinity Logistics,2.021124e+11
1,309187241.0,9010143874,0405510801,0284-2025320-3895,Shipped,Customer Orders,HEYL,TARGET FDC 3895,"Jan 29, 2024 12:00:00 AM",4219650.0,"Jan 29, 2024 11:00:00 AM",01/29/24 08:47,DROP,2203,NaN,1,NaN,NaN,NaN,2.021124e+11
2,309187222.0,9010143871,0405510802,0284-3960824-3895,Shipped,Customer Orders,HEYL,TARGET FDC 3895,"Jan 29, 2024 12:00:00 AM",4219652.0,"Jan 29, 2024 11:00:00 AM",01/26/24 22:06,DROP,22072,NaN,1,NaN,NaN,NaN,2.021124e+11
3,309187196.0,9010143870,0405510804,0284-2156124-3895,Shipped,Customer Orders,HEYL,TARGET FDC 3895,"Jan 29, 2024 12:00:00 AM",4219656.0,"Jan 29, 2024 11:00:00 AM",01/27/24 03:52,DROP,18050,NaN,1,NaN,NaN,NaN,2.021124e+11
4,309187223.0,9010143869,0405510806,0284-9664329-3895,Shipped,Customer Orders,HEYL,TARGET FDC 3895,"Jan 29, 2024 12:00:00 AM",4219658.0,"Jan 29, 2024 11:00:00 AM",01/26/24 21:54,DROP,24146,NaN,1,NaN,NaN,NaN,2.021124e+11


In [458]:
order_report.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46035 entries, 0 to 46034
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Ref Shipment Nbr         45918 non-null  float64
 1   Shipment #               46035 non-null  object 
 2   SAP Delivery # (Order#)  46035 non-null  object 
 3   PO #                     46025 non-null  object 
 4   Order Status             46035 non-null  object 
 5   Order Type               46035 non-null  object 
 6   Carrier                  46029 non-null  object 
 7   Ship To                  46035 non-null  object 
 8   Order Pick Up Date       46035 non-null  object 
 9   Appointment              38005 non-null  float64
 10  Appointment Date         38005 non-null  object 
 11  Check In DateTime        44740 non-null  object 
 12  Appointment Type         38000 non-null  object 
 13  Trailer                  45583 non-null  object 
 14  LOAD_OF_THE_DAY       

In [459]:
order_report.isnull().sum()

Ref Shipment Nbr             117
Shipment #                     0
SAP Delivery # (Order#)        0
PO #                          10
Order Status                   0
Order Type                     0
Carrier                        6
Ship To                        0
Order Pick Up Date             0
Appointment                 8030
Appointment Date            8030
Check In DateTime           1295
Appointment Type            8035
Trailer                      452
LOAD_OF_THE_DAY            44667
POOL_SEQ#                      0
Trailer Location           41733
Trailer Status             41603
Appointment Notes          32011
Wave #                      1531
dtype: int64

In [460]:
duplicated_sums = order_report['Shipment #'].duplicated().sum()
print (duplicated_sums)

5217


In [461]:
order_report['Appointment Date'] = pd.to_datetime(order_report['Appointment Date'])
order_report_sorted = order_report.sort_values(by=['Shipment #', 'Appointment Date'], ascending=[True, False])
order_report = order_report_sorted.drop_duplicates(subset='Shipment #', keep='first')

In [462]:
duplicated_sums = order_report['Shipment #'].duplicated().sum()
print (duplicated_sums)

0


In [463]:
print(order_report.columns)

Index(['Ref Shipment Nbr', 'Shipment #', 'SAP Delivery # (Order#)', 'PO #',
       'Order Status', 'Order Type', 'Carrier', 'Ship To',
       'Order Pick Up Date', 'Appointment', 'Appointment Date',
       'Check In DateTime', 'Appointment Type', 'Trailer', 'LOAD_OF_THE_DAY',
       'POOL_SEQ#', 'Trailer Location', 'Trailer Status', 'Appointment Notes',
       'Wave #'],
      dtype='object')


In [464]:
columns_to_keep = ['Shipment #', 'SAP Delivery # (Order#)', 'Appointment Date', 'Carrier', 'Appointment Type']
order_report = order_report.drop(columns=order_report.columns.difference(columns_to_keep))

order_report.head()

,Shipment #,SAP Delivery # (Order#),Carrier,Appointment Date,Appointment Type
29,9010143843,0405510328,CPU,2024-01-29 18:00:00,DROP
160,9010143844,0405510495,CPU,2024-01-31 16:00:00,LIVE
0,9010143845,0405510588,CPU,2024-01-29 10:00:00,LIVE
1111,9010143846,0405510727,CPU,2024-02-09 19:00:00,DROP
201,9010143847,0405510737,CPU,2024-02-01 15:00:00,DROP


In [465]:
unique_values = order_report['Appointment Type'].unique()
print(unique_values)

['DROP' 'LIVE' nan]


In [466]:
order_report.isnull().sum()

Shipment #                    0
SAP Delivery # (Order#)       0
Carrier                       4
Appointment Date           7994
Appointment Type           7999
dtype: int64

In [467]:
order_report = order_report.dropna()

In [468]:
order_report.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32816 entries, 29 to 37965
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Shipment #               32816 non-null  object        
 1   SAP Delivery # (Order#)  32816 non-null  object        
 2   Carrier                  32816 non-null  object        
 3   Appointment Date         32816 non-null  datetime64[ns]
 4   Appointment Type         32816 non-null  object        
dtypes: datetime64[ns](1), object(4)
memory usage: 1.5+ MB


In [469]:
order_report.rename(columns={'Shipment #': 'Shipment Num', 'SAP Delivery # (Order#)': 'Order Num', 'Appointment Date': 'Appointment DateTime', 'Appointment Type': 'Visit Type'}, inplace=True)

In [470]:
order_report.head()

,Shipment Num,Order Num,Carrier,Appointment DateTime,Visit Type
29,9010143843,0405510328,CPU,2024-01-29 18:00:00,DROP
160,9010143844,0405510495,CPU,2024-01-31 16:00:00,LIVE
0,9010143845,0405510588,CPU,2024-01-29 10:00:00,LIVE
1111,9010143846,0405510727,CPU,2024-02-09 19:00:00,DROP
201,9010143847,0405510737,CPU,2024-02-01 15:00:00,DROP


## Trailer Activity Report

In [471]:
trailer_report.head()

,CHECKIN DATE TIME,CHECKOUT DATE TIME,Trailer time on yard in hours,TRAILER,Trailer Type (Std or Reef),CARRIER,APPOINTMENT,VISIT TYPE,TRAILER STATUS,TASK TYPE,ACTIVITY TYPE,Date/Time,YARD SLOT,DOCK DOOR,DRIVER,ACTIVITY SOURCE,SHIPMENT_ID,ASN_ID,PO #
0,"Feb 12, 2024 4:01:00 PM","Feb 12, 2024 11:20:37 PM",7,220,Reefer,CPU,NaN,Live Unload,IB Loaded,NaN,CHECKED IN,02/12/24 16:01,NaN,DR230,NaN,172691,NaN,201364813.0,4702106224
1,"Feb 1, 2024 1:07:00 AM","Feb 3, 2024 2:27:06 PM",61,19035,Shuttle,HEY2,NaN,Drop Unload,IB Loaded,Move,TRAILER MOVED,02/03/24 00:37,NaN,DR196,NaN,146501,NaN,201365001.0,4702106353
2,"Feb 1, 2024 9:05:00 PM","Feb 10, 2024 4:08:05 AM",199,17091,Shuttle,HEY2,42111970.0,Drop Unload,IB Loaded,NaN,CHECKED IN,02/01/24 21:05,[R]B-124,124,NaN,169398,NaN,201365186.0,4702106525
3,"Feb 3, 2024 8:32:00 AM","Feb 13, 2024 5:04:40 AM",237,17007,Reefer,HEY2,NaN,Live Unload,Empty,NaN,CLOSED,02/13/24 05:02,NaN,DR224,NaN,170158,NaN,201365264.0,4702106599
4,"Feb 13, 2024 8:05:00 AM","Feb 13, 2024 9:49:33 AM",2,2022,Reefer,CPU,42113722.0,Drop Unload,IB Loaded,NaN,CHECKED IN,02/13/24 08:06,NaN,DR214,NaN,172691,NaN,60569659.0,4509844863


In [472]:
print(trailer_report.columns)

Index(['CHECKIN DATE TIME', 'CHECKOUT DATE TIME',
       'Trailer time on yard in hours', 'TRAILER',
       'Trailer Type (Std or Reef)', 'CARRIER', 'APPOINTMENT', 'VISIT TYPE',
       'TRAILER STATUS', ' TASK TYPE', 'ACTIVITY TYPE ', 'Date/Time',
       'YARD SLOT', 'DOCK DOOR', 'DRIVER', 'ACTIVITY SOURCE', 'SHIPMENT_ID',
       'ASN_ID', 'PO #'],
      dtype='object')


In [473]:
trailer_report = trailer_report[trailer_report['ACTIVITY TYPE '] == 'CLOSED']

In [474]:
columns_to_keep = ['CHECKIN DATE TIME', 'CHECKOUT DATE TIME', 'Date/Time', 'SHIPMENT_ID', 'Date/Time', ]
trailer_report = trailer_report.drop(columns=trailer_report.columns.difference(columns_to_keep))

In [475]:
trailer_report.head()

,CHECKIN DATE TIME,CHECKOUT DATE TIME,Date/Time,SHIPMENT_ID
3,"Feb 3, 2024 8:32:00 AM","Feb 13, 2024 5:04:40 AM",02/13/24 05:02,NaN
5,"Feb 8, 2024 2:12:00 PM","Feb 8, 2024 7:49:40 PM",02/08/24 17:56,NaN
6,"Feb 16, 2024 4:10:00 PM","Feb 21, 2024 12:30:34 PM",02/20/24 15:41,NaN
8,"Feb 17, 2024 10:34:00 AM","Feb 17, 2024 2:30:14 PM",02/17/24 14:07,NaN
9,"Feb 5, 2024 5:25:00 AM","Feb 5, 2024 11:53:10 AM",02/05/24 08:05,NaN


In [476]:
trailer_report = trailer_report.dropna()

In [477]:
trailer_report.isnull().sum()

CHECKIN DATE TIME     0
CHECKOUT DATE TIME    0
Date/Time             0
SHIPMENT_ID           0
dtype: int64

In [478]:
row_count = len(trailer_report)
print(row_count)

47183


In [479]:
duplicated_sums = trailer_report['SHIPMENT_ID'].duplicated().sum()
print(duplicated_sums)

9026


In [481]:
trailer_report['Date/Time'] = pd.to_datetime(trailer_report['Date/Time'])
trailer_report = trailer_report.sort_values(by=['SHIPMENT_ID', 'Date/Time'], ascending=[True, False])

trailer_report = trailer_report.drop_duplicates(subset='SHIPMENT_ID', keep='first')

row_count = len(trailer_report)
print(row_count)

38157


In [482]:
trailer_report.rename(columns={'CHECKIN DATE TIME': 'Checkin DateTime', 'CHECKOUT DATE TIME': 'Checkout DateTime', 'SHIPMENT_ID': 'Shipment Num', 'Date/Time': 'Loaded DateTime'}, inplace=True)

In [485]:
trailer_report['Checkin DateTime'] = pd.to_datetime(trailer_report['Checkin DateTime'])
trailer_report['Checkout DateTime'] = pd.to_datetime(trailer_report['Checkout DateTime'])

/var/folders/9x/s44ysdgx3ydcqwy4xgb366d40000gn/T/ipykernel_98957/626782885.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trailer_report['Checkin DateTime'] = pd.to_datetime(trailer_report['Checkin DateTime'])
/var/folders/9x/s44ysdgx3ydcqwy4xgb366d40000gn/T/ipykernel_98957/626782885.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  trailer_report['Checkout DateTime'] = pd.to_datetime(trailer_report['Checkout DateTime'])


In [486]:
trailer_report.head()

,Checkin DateTime,Checkout DateTime,Loaded DateTime,Shipment Num
256144,2024-01-29 23:11:00,2024-01-30 18:29:02,2024-01-30 18:28:00,9010143843
162604,2024-01-31 22:49:00,2024-02-01 14:09:30,2024-02-01 14:09:00,9010143844
125823,2024-01-29 11:11:00,2024-01-29 16:31:18,2024-01-29 16:30:00,9010143845
53956,2024-02-10 18:44:00,2024-02-11 00:01:24,2024-02-11 00:01:00,9010143846
238488,2024-02-02 02:06:00,2024-02-03 12:32:46,2024-02-02 07:03:00,9010143847
